In [1]:
!pip install cyipopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cyipopt: filename=cyipopt-1.5.0-cp311-cp311-linux_x86_64.whl size=852879 sha256=a89929c15885a5a95058081e4a832f0b21764470fdbef549adf9a21bb5ea373b
  Stored in directory: /root/.cache/pip/wheels/5c/7e/0c/efed40804bdab977ee1bac731a5f583397e36abef25e65ca5d
Successfully built cyipopt

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install pyomo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 9.6 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 8.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pyomo download-extensions

INFO: As of February 9, 2023, AMPL GSL can no longer be downloaded
through download-extensions. Visit https://portal.ampl.com/             to
download the AMPL GSL binaries.
INFO: Fetching GJH from https://netlib.org/ampl/student/linux/gjh.gz and
installing it to /root/.pyomo/bin/gjh
INFO: ...downloaded 113965 bytes
INFO: ...wrote 220564 bytes
INFO: Fetching MC++ from https://github.com/omega-icl/mcpp/archive/master.zip
and installing it to /root/.pyomo/src/mcpp
INFO: ...downloaded 2836791 bytes
INFO: Finished downloading Pyomo extensions.
INFO: The following extensions were downloaded:
    [ OK ]  gjh
    [ OK ]  mcpp


In [4]:
!pyomo build-extensions


**** Building AMPL External function demo library ****
-- The C compiler identification is GNU 10.2.1
-- The CXX compiler identification is GNU 10.2.1
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found PkgConfig: /usr/bin/pkg-config (found version "0.29.2") 
-- Configuring done
-- Generating done
-- Build files have been written to: /tmp/tmpqumpz5nn
Scanning dependencies of target ampl_asl
[ 11%] Creating directories for 'ampl_asl'
[ 22%] Performing download step (git clone) for 'ampl_asl'
Cloning into 'ampl_asl'...
Note: switching to '9fb7cb8e4f68ed1c3bc066d191e63698b7d7d1d2'.

You are in 'detached HEAD' state. You can look

In [5]:
!pyomo help --solvers


Pyomo Solvers and Solver Managers
---------------------------------
Pyomo uses 'solver managers' to execute 'solvers' that perform
optimization and other forms of model analysis.  A solver directly
executes an optimizer, typically using an executable found on the
user's PATH environment.  Solver managers support a flexible mechanism
for asynchronously executing solvers either locally or remotely.  The
following solver managers are available in Pyomo:

    neos       Asynchronously execute solvers on the NEOS server
    serial     Synchronously execute solvers locally

If no solver manager is specified, Pyomo uses the serial solver
manager to execute solvers locally.  The neos solver manager is used
to execute solvers on the NEOS optimization server.


Serial Solver Interfaces
------------------------
The serial manager supports the following solver interfaces:

    appsi_cbc                   Automated persistent interface to Cbc
    appsi_cplex                 Automated persistent in

# Test of pyomo

https://pyomo.readthedocs.io/en/6.8.0/contributed_packages/mindtpy.html

In [6]:
from pyomo.environ import ConcreteModel, Var, Binary, Constraint, Objective, SolverFactory, log, minimize

In [7]:
SolverFactory("glpk").available()

True

In [8]:
SolverFactory("cyipopt").available()

True

In [9]:
model = ConcreteModel()

model.x = Var(bounds=(1.0,10.0),initialize=5.0)
model.y = Var(within=Binary)

model.c1 = Constraint(expr=(model.x-4.0)**2 - model.x <= 50.0*(1-model.y))
model.c2 = Constraint(expr=model.x*log(model.x)+5.0 <= 50.0*(model.y))

model.objective = Objective(expr=model.x, sense=minimize)

In [10]:
#SolverFactory("mindtpy").solve(model, mip_solver="glpk", nlp_solver="ipopt") 
sol = SolverFactory("mindtpy").solve(model, mip_solver="glpk", nlp_solver="cyipopt") 

/usr/local/lib/python3.11/site-packages/pyomo/contrib/mindtpy/algorithm_base_class.py:531: RuntimeWarning: invalid value encountered in scalar divide
  self.rel_gap = self.abs_gap / (abs(self.primal_bound) + 1e-10)


In [11]:
sol.keys()

['Problem', 'Solver', 'Solution']

In [12]:
for key, item in sol.items():
    print(f"Category '{key}' has the following options: \n{item.keys()}\n\n")

Category 'Problem' has the following options: 
['Name', 'Lower bound', 'Upper bound', 'Number of objectives', 'Number of constraints', 'Number of variables', 'Number of binary variables', 'Number of integer variables', 'Number of continuous variables', 'Number of nonzeros', 'Sense', 'Number of disjunctions']


Category 'Solver' has the following options: 
['Name', 'Status', 'Return code', 'Message', 'User time', 'System time', 'Wallclock time', 'Termination condition', 'Termination message', 'Statistics', 'Timing', 'Iterations', 'Num infeasible nlp subproblem', 'Best solution found time', 'Primal integral', 'Dual integral', 'Primal dual gap integral']


Category 'Solution' has the following options: 
['Gap', 'Status', 'Message', 'Problem', 'Objective', 'Variable', 'Constraint']




In [13]:
for kk in sol['Solution']:
    print(kk)

Status: unknown
Objective: No values
Variable: No values
Constraint: No values



In [14]:
sol

{'Problem': [{'Name': 'unknown', 'Lower bound': 2.43844718719117, 'Upper bound': np.float64(2.4384471872707754), 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 2, 'Number of binary variables': 1, 'Number of integer variables': 0, 'Number of continuous variables': 1, 'Number of nonzeros': None, 'Sense': 'minimize', 'Number of disjunctions': 0}], 'Solver': [{'Name': 'MindtPyOA', 'Status': 'ok', 'Message': None, 'User time': 0.32538530899910256, 'Wallclock time': 0.32538530899910256, 'Termination condition': 'optimal', 'Termination message': None, 'Timing': Bunch(Call after main solve = 1.8904996977653354e-05, Call after subproblem solve = 9.7439915407449e-06, Call before subproblem solve = 7.5699936132878065e-06, OA cut generation = 0.001857228999142535, feasibility subproblem = 0.03912882700387854, fixed subproblem = 0.03912669999408536, initialization = 0.18830141799844569, main = 0.028370097003062256, main loop = 0.11165958199853776, main_timer_start_tim

In [15]:
model.objective.display()

objective : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 2.4384471872707754


In [16]:
model.display()

Model unknown

  Variables:
    x : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :   1.0 : 2.4384471872707754 :  10.0 : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 2.4384471872707754

  Constraints:
    c1 : Size=1
        Key  : Lower : Body                    : Upper
        None :  None : -3.2822233819729263e-10 :   0.0
    c2 : Size=1
        Key  : Lower : Body               : Upper
        None :  None : -42.82646220852581 :   0.0


In [17]:
model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :   1.0 : 2.4384471872707754 :  10.0 : False : False :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :     1 : False : False : Binary

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize :          x

2 Constraint Declarations
    c1 : Size=1, Index=None, Active=True
        Key  : Lower : Body                            : Upper : Active
        None :  -Inf : (x - 4.0)**2 - x - 50.0*(1 - y) :   0.0 :   True
    c2 : Size=1, Index=None, Active=True
        Key  : Lower : Body                    : Upper : Active
        None :  -Inf : x*log(x) + 5.0 - 50.0*y :   0.0 :   True

5 Declarations: x y c1 c2 objective


# TUM MIP w/ pyomo tutorial

https://www.math.cit.tum.de/fileadmin/w00ccg/math/personen/discrete_math/Andreas_Wiese/MIP-coding-cookbook.pdf

## Example for a simple MIP (section 1.1)

In [27]:
import pyomo.environ as pyo

In [30]:
model = pyo.ConcreteModel()

model.x1 = pyo.Var(within=pyo.PositiveIntegers)
model.x2 = pyo.Var(within=pyo.PositiveIntegers)

model.c1 = pyo.Constraint(expr=2*model.x1 + 3*model.x2 <= 18)
model.c2 = pyo.Constraint(expr=4*model.x1 + 3*model.x2 <= 24)

model.objective = pyo.Objective(expr=model.x1 + model.x2, sense=pyo.maximize)

In [31]:
solver = pyo.SolverFactory("glpk")  # or "cbc", "gurobi" if available
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 7.0, 'Upper bound': 7.0, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 2, 'Number of nonzeros': 4, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.00399327278137207}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [32]:
model.display()

Model unknown

  Variables:
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     1 :   3.0 :  None : False : False : PositiveIntegers
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     1 :   4.0 :  None : False : False : PositiveIntegers

  Objectives:
    objective : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   7.0

  Constraints:
    c1 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 18.0 :  18.0
    c2 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 24.0 :  24.0


## Solving a simple MIP (section 4)

In [100]:
import pyomo.environ as pyo

m = pyo.ConcreteModel()

m.x1 = pyo.Var(domain=pyo.NonNegativeReals)
m.x2 = pyo.Var(domain=pyo.NonPositiveReals)
m.x3 = pyo.Var(domain=pyo.NonNegativeIntegers)
m.x4 = pyo.Var(domain=pyo.Reals)
m.x5 = pyo.Var(domain=pyo.Binary)
m.y = pyo.Var([1,2], domain=pyo.NonNegativeReals)

m.constr1 = pyo.Constraint(expr=2*m.x1 + 3*m.x2 -4*m.x3 +m.x4 +m.x5 + m.y[1] <= 180)
m.constr2 = pyo.Constraint(expr=4*m.x1 + 3*m.x2 -4*m.x4 +m.x5 - m.y[2] >= -240)
m.constr3 = pyo.Constraint(expr=5*m.x1 + 2*m.x2 -5*m.x4 +m.x5 == 120)

m.obj = pyo.Objective(expr=m.x1 + 5*m.x2 + 3*m.x3 - m.x5, sense = pyo.minimize)

solver=pyomo.opt.SolverFactory('glpk')
results=solver.solve(m)

In [105]:
m.x1.display()

x1 : Size=1, Index=None
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   0.0 :  None : False : False : NonNegativeReals


In [101]:
m.display()

Model unknown

  Variables:
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    x2 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :  None : -240.142857142857 :     0 : False : False : NonPositiveReals
    x3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    x4 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :  None : -119.857142857143 :  None : False : False :  Reals
    x5 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :     1 : False : False : Binary
    y : Size=2, Index={1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : Fal

# Permutation matrix

In [7]:
import numpy as np
import random

dim = 3
A = np.random.randint(0, 2*dim**2, size=dim**2, dtype=int).reshape(dim, dim)

p = list(range(dim))
random.shuffle(p)
P1 = np.zeros((dim, dim), dtype=int)
for row, col in enumerate(p):
    P1[row, col] = 1

p = list(range(dim))
random.shuffle(p)
P2 = np.zeros((dim, dim), dtype=int)
for row, col in enumerate(p):
    P2[row, col] = 1

A_perm = np.dot(np.dot(P1, A), P2)

A, P1, P2, A_perm

(array([[ 4, 14, 13],
        [14,  9,  9],
        [ 1, 16,  8]]),
 array([[0, 1, 0],
        [0, 0, 1],
        [1, 0, 0]]),
 array([[0, 1, 0],
        [0, 0, 1],
        [1, 0, 0]]),
 array([[ 9, 14,  9],
        [ 8,  1, 16],
        [13,  4, 14]]))

In [8]:
import pyomo.environ as pyo

# Pyomo Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(dim)  # Rows
model.J = pyo.RangeSet(dim)  # Columns

# Variables (Binary Permutation Matrices)
model.P1 = pyo.Var(model.I, model.J, within=pyo.Binary)
model.P2 = pyo.Var(model.I, model.J, within=pyo.Binary)

# Constraints: Row and Column Sums = 1 for P1 and P2
model.row_sum_p1 = pyo.Constraint(model.I, rule=lambda model, i: sum(model.P1[i, j] for j in model.J) == 1)
model.col_sum_p1 = pyo.Constraint(model.J, rule=lambda model, j: sum(model.P1[i, j] for i in model.I) == 1)
model.row_sum_p2 = pyo.Constraint(model.I, rule=lambda model, i: sum(model.P2[i, j] for j in model.J) == 1)
model.col_sum_p2 = pyo.Constraint(model.J, rule=lambda model, j: sum(model.P2[i, j] for i in model.I) == 1)

# Note: pyomo indexes vary from 1 to N 
def compute_C(model, i, j):
    return sum(A[i-1, k-1] * model.P2[k, j] for k in model.J)
model.C = pyo.Expression(model.I, model.J, rule=compute_C)

def compute_D(model, i, j):
    return sum(model.P1[k, i] * A_perm[k-1, j-1] for k in model.J)
model.D = pyo.Expression(model.I, model.J, rule=compute_D)

# Constraint: C <= D
model.c1 = pyo.Constraint(model.I, model.J, rule=lambda model, i, j: model.C[i, j] <= model.D[i, j])
# Constraint: C >= D
model.c2 = pyo.Constraint(model.I, model.J, rule=lambda model, i, j: model.C[i, j] >= model.D[i, j])

model.obj = pyo.Objective(expr=0, sense=pyo.minimize)

In [9]:
solver = pyo.SolverFactory("glpk")  # or "cbc", "gurobi" if available
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.0, 'Upper bound': 0.0, 'Number of objectives': 1, 'Number of constraints': 30, 'Number of variables': 19, 'Number of nonzeros': 144, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.025197505950927734}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [38]:
def build_matrix(pyomo_matrix):
    values = pyomo_matrix.get_values()
    pp = np.zeros((dim,dim), dtype=int)
    for idx, val in values.items():
        i, j = idx[0]-1, idx[1]-1
        pp[i,j] = val
    return pp

In [39]:
build_matrix(model.P1)

array([[0, 1, 0],
       [0, 0, 1],
       [1, 0, 0]])

In [41]:
P1

array([[0, 1, 0],
       [0, 0, 1],
       [1, 0, 0]])

In [40]:
build_matrix(model.P2)

array([[0, 1, 0],
       [0, 0, 1],
       [1, 0, 0]])

In [42]:
P2

array([[0, 1, 0],
       [0, 0, 1],
       [1, 0, 0]])